## Imports

In [13]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [14]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=2)

# Get the date one week ago
one_week_ago = today - timedelta(days=8)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

May. 17th May. 25th


In [15]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [16]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [17]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [18]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [19]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,May-21-2025,Condo/Co-op,1083 Exeter E,Boca Raton,FL,33434.0,175000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1083-Exet...,Beaches MLS,RX-11078992,N,Y,26.385037,-80.17811


In [20]:
print('Input county name:')
county = input()

Input county name:
Palm Beach


In [21]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [22]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [23]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [24]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-05-21    23
2025-05-20    23
2025-05-23    23
2025-05-22    16
Name: count, dtype: int64

In [25]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [26]:
df_top_ten = df2.head(10)

In [27]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
15,PAST SALE,2025-05-22,Condo/Co-op,150 N Ocean Blvd #403,Palm Beach,FL,33480.0,7900000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Beaches MLS,RX-11076066,N,Y,26.720386,-80.034950
53,PAST SALE,2025-05-21,Condo/Co-op,200 SE Mizner Blvd #507,Boca Raton,FL,33432.0,3501000.0,2.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11049060,N,Y,26.347715,-80.083263
56,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 201n,Palm Beach,FL,33480.0,3100000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11048433,N,Y,26.623992,-80.037641
65,PAST SALE,2025-05-21,Condo/Co-op,3730 N Ocean Dr Unit 11d,Singer Island,FL,33404.0,2995000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Beaches MLS,RX-11019236,N,Y,26.791736,-80.033375
11,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 402s,Palm Beach,FL,33480.0,2190000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11068446,N,Y,26.623992,-80.037641
34,PAST SALE,2025-05-20,Condo/Co-op,131 Ocean Grande Blvd #808,Jupiter,FL,33477.0,2050000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,Beaches MLS,RX-11068190,N,Y,26.919651,-80.066425
78,PAST SALE,2025-05-21,Condo/Co-op,1400 S Ocean Blvd Unit 104n,Boca Raton,FL,33432.0,1555000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Beaches MLS,RX-11018830,N,Y,26.363561,-80.068653
2,PAST SALE,2025-05-20,Condo/Co-op,102 Palm Bay Cir Unit C,Palm Beach Gardens,FL,33418.0,1275000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Beaches MLS,RX-11063852,N,Y,26.817417,-80.107623
10,PAST SALE,2025-05-20,Condo/Co-op,104 Chambord Ter Ter,Palm Beach Gardens,FL,33410.0,1200000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Beaches MLS,RX-11067653,N,Y,26.866687,-80.094542
13,PAST SALE,2025-05-21,Condo/Co-op,3000 N Ocean Dr Unit 23-C,Singer Island,FL,33404.0,1150000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3000-N...,Beaches MLS,RX-11078403,N,Y,26.786861,-80.033252


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [28]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [29]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [30]:
response_df = pd.DataFrame(response_list)

In [31]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [32]:
df_top_ten = merged_df

In [33]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-05-22,Condo/Co-op,150 N Ocean Blvd #403,Palm Beach,FL,33480.0,7900000.0,3.0,4.5,...,-80.034950,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Christopher Leavitt,Douglas Elliman,None,None,Lawrence Moens,Lawrence A. Moens Associates,None,None
1,PAST SALE,2025-05-21,Condo/Co-op,200 SE Mizner Blvd #507,Boca Raton,FL,33432.0,3501000.0,2.0,3.5,...,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Rochelle LeCavalier,Douglas Elliman,None,None
2,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 201n,Palm Beach,FL,33480.0,3100000.0,2.0,2.5,...,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Thor Brown,Compass Florida LLC (PB),None,None,Ashley Copeland,Brown Harris Stevens of PB,None,None
3,PAST SALE,2025-05-21,Condo/Co-op,3730 N Ocean Dr Unit 11d,Singer Island,FL,33404.0,2995000.0,3.0,3.0,...,-80.033375,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Judy Ballard,Illustrated Properties LLC (Co,None,None,John Pinkham,Exceptional Properties,None,None
4,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 402s,Palm Beach,FL,33480.0,2190000.0,2.0,2.0,...,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Scott Gordon,Douglas Elliman,None,None
5,PAST SALE,2025-05-20,Condo/Co-op,131 Ocean Grande Blvd #808,Jupiter,FL,33477.0,2050000.0,3.0,3.5,...,-80.066425,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,Mary Martire,Illustrated Properties LLC (Jupiter),None,None,Jeremy Sitton,Sutter & Nugent LLC,None,None
6,PAST SALE,2025-05-21,Condo/Co-op,1400 S Ocean Blvd Unit 104n,Boca Raton,FL,33432.0,1555000.0,2.0,2.5,...,-80.068653,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Bonnie Heatzig,Compass Florida LLC,None,None,Lisa Margolin,Realty Home Advisors Inc,None,None
7,PAST SALE,2025-05-20,Condo/Co-op,102 Palm Bay Cir Unit C,Palm Beach Gardens,FL,33418.0,1275000.0,3.0,3.0,...,-80.107623,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Todd Blair,Palm Beach Coastal Realty LLC,None,None,Laura Cole,Illustrated Properties LLC (Co,None,None
8,PAST SALE,2025-05-20,Condo/Co-op,104 Chambord Ter Ter,Palm Beach Gardens,FL,33410.0,1200000.0,2.0,2.5,...,-80.094542,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Richard Wood,Frankel Ball Realty LLC,Wendy Moyer,Frankel Ball Realty LLC,Richard Wood,Frankel Ball Realty LLC,None,None
9,PAST SALE,2025-05-21,Condo/Co-op,3000 N Ocean Dr Unit 23-C,Singer Island,FL,33404.0,1150000.0,2.0,2.0,...,-80.033252,https://www.redfin.com/FL/Riviera-Beach/3000-N...,Matthew Brannon,Illustrated Properties LLC (Co,None,None,Susan Bennett,The Keyes Company (PBG),Leah Tatum,The Keyes Company (PBG)


## Current Week's Values

In [34]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

85


In [35]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$570,234


In [36]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$329


In [37]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$48,469,925


In [38]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           85              570234          329                     48469925.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [39]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
97
-----------
Input Previous Week Condo Average Sales Price:
717106
-----------
Input Previous Week Condo Average PSF:
391
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
69559250.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [40]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [41]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [42]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [43]:
pd.set_option('display.max_columns',None)

In [44]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-22,Condo/Co-op,150 N Ocean Blvd #403,Palm Beach,FL,33480.0,7900000.0,3.0,4.5,Number One North Ocean Condo,3036.0,NaN,1980.0,NaN,2602.0,6566.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Beaches MLS,RX-11076066,N,Y,26.720386,-80.03495,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Christopher Leavitt,Douglas Elliman,None,None,Lawrence Moens,Lawrence A. Moens Associates,None,None,1,orange


In [45]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-22,Condo/Co-op,150 N Ocean Blvd #403,Palm Beach,FL,33480.0,7900000.0,3.0,4.5,Number One North Ocean Condo,3036.0,NaN,1980.0,NaN,2602.0,6566.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Beaches MLS,RX-11076066,N,Y,26.720386,-80.034950,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Christopher Leavitt,Douglas Elliman,None,None,Lawrence Moens,Lawrence A. Moens Associates,None,None,1,orange
1,PAST SALE,2025-05-21,Condo/Co-op,200 SE Mizner Blvd #507,Boca Raton,FL,33432.0,3501000.0,2.0,3.5,Alina Boca Raton Condo,2713.0,NaN,2021.0,NaN,1290.0,3216.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11049060,N,Y,26.347715,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Rochelle LeCavalier,Douglas Elliman,None,None,2,blue
2,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 201n,Palm Beach,FL,33480.0,3100000.0,2.0,2.5,Beach Point Condo,2137.0,NaN,1979.0,NaN,1451.0,2799.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11048433,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Thor Brown,Compass Florida LLC (PB),None,None,Ashley Copeland,Brown Harris Stevens of PB,None,None,3,blue
3,PAST SALE,2025-05-21,Condo/Co-op,3730 N Ocean Dr Unit 11d,Singer Island,FL,33404.0,2995000.0,3.0,3.0,Vistablue,2686.0,NaN,2018.0,NaN,1115.0,3645.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Beaches MLS,RX-11019236,N,Y,26.791736,-80.033375,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Judy Ballard,Illustrated Properties LLC (Co,None,None,John Pinkham,Exceptional Properties,None,None,4,blue
4,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 402s,Palm Beach,FL,33480.0,2190000.0,2.0,2.0,Beach Point Cond Decl Filed 5-31-79,1900.0,NaN,1979.0,NaN,1153.0,2149.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11068446,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Scott Gordon,Douglas Elliman,None,None,5,blue
5,PAST SALE,2025-05-20,Condo/Co-op,131 Ocean Grande Blvd #808,Jupiter,FL,33477.0,2050000.0,3.0,3.5,Jupiter Ocean Grande 1 Condo,2376.0,NaN,2001.0,NaN,863.0,2283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,Beaches MLS,RX-11068190,N,Y,26.919651,-80.066425,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,Mary Martire,Illustrated Properties LLC (Jupiter),None,None,Jeremy Sitton,Sutter & Nugent LLC,None,None,6,blue
6,PAST SALE,2025-05-21,Condo/Co-op,1400 S Ocean Blvd Unit 104n,Boca Raton,FL,33432.0,1555000.0,2.0,2.5,Addison on The Ocean,2156.0,NaN,1986.0,NaN,721.0,3348.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Beaches MLS,RX-11018830,N,Y,26.363561,-80.068653,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Bonnie Heatzig,Compass Florida LLC,None,None,Lisa Margolin,Realty Home Advisors Inc,None,None,7,blue
7,PAST SALE,2025-05-20,Condo/Co-op,102 Palm Bay Cir Unit C,Palm Beach Gardens,FL,33418.0,1275000.0,3.0,3.0,Palm Bay Club Condo,2483.0,43560.0,2003.0,NaN,513.0,1637.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Beaches MLS,RX-11063852,N,Y,26.817417,-80.107623,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Todd Blair,Palm Beach Coastal Realty LLC,None,None,Laura Cole,Illustrated Properties LLC (Co,None,None,8,blue
8,PAST SALE,2025-05-20,Condo/Co-op,104 Chambord Ter Ter,Palm Beach Gardens,FL,33410.0,1200

In [47]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [48]:
m.save('index.html')

## Data snagger

In [49]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [50]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-22,Condo/Co-op,150 N Ocean Blvd #403,Palm Beach,FL,33480.0,7900000.0,3.0,4.5,Number One North Ocean Condo,3036.0,NaN,1980.0,NaN,2602.0,6566.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Beaches MLS,RX-11076066,N,Y,26.720386,-80.03495,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Christopher Leavitt,Douglas Elliman,None,None,Lawrence Moens,Lawrence A. Moens Associates,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [51]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_05272025


# CREATE TEMPLATE 

In [52]:
muni_set = set(df_top_ten['CITY'])

In [53]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [54]:
df_top_ten.reset_index(inplace=True,drop=True)

In [55]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [56]:
top_sale

'Number One North Ocean Condo at 150 N Ocean Blvd #403 in Palm Beach'

In [57]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [58]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [59]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-22,Condo/Co-op,150 N Ocean Blvd #403,Palm Beach,FL,33480.0,7900000.0,3.0,4.5,Number One North Ocean Condo,3036.0,NaN,1980.0,NaN,2602.0,6566.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Beaches MLS,RX-11076066,N,Y,26.720386,-80.034950,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Christopher Leavitt,Douglas Elliman,None,None,Lawrence Moens,Lawrence A. Moens Associates,None,None,1,orange,Number One North Ocean Condo at 150 N Ocean Bl...
1,PAST SALE,2025-05-21,Condo/Co-op,200 SE Mizner Blvd #507,Boca Raton,FL,33432.0,3501000.0,2.0,3.5,Alina Boca Raton Condo,2713.0,NaN,2021.0,NaN,1290.0,3216.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11049060,N,Y,26.347715,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Rochelle LeCavalier,Douglas Elliman,None,None,2,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
2,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 201n,Palm Beach,FL,33480.0,3100000.0,2.0,2.5,Beach Point Condo,2137.0,NaN,1979.0,NaN,1451.0,2799.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11048433,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Thor Brown,Compass Florida LLC (PB),None,None,Ashley Copeland,Brown Harris Stevens of PB,None,None,3,blue,Beach Point Condo at 2660 S Ocean Blvd Unit 20...
3,PAST SALE,2025-05-21,Condo/Co-op,3730 N Ocean Dr Unit 11d,Singer Island,FL,33404.0,2995000.0,3.0,3.0,Vistablue,2686.0,NaN,2018.0,NaN,1115.0,3645.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Beaches MLS,RX-11019236,N,Y,26.791736,-80.033375,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Judy Ballard,Illustrated Properties LLC (Co,None,None,John Pinkham,Exceptional Properties,None,None,4,blue,Vistablue at 3730 N Ocean Dr Unit 11d in Singe...
4,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 402s,Palm Beach,FL,33480.0,2190000.0,2.0,2.0,Beach Point Cond Decl Filed 5-31-79,1900.0,NaN,1979.0,NaN,1153.0,2149.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11068446,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Scott Gordon,Douglas Elliman,None,None,5,blue,Beach Point Cond Decl Filed 5-31-79 at 2660 S ...
5,PAST SALE,2025-05-20,Condo/Co-op,131 Ocean Grande Blvd #808,Jupiter,FL,33477.0,2050000.0,3.0,3.5,Jupiter Ocean Grande 1 Condo,2376.0,NaN,2001.0,NaN,863.0,2283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,Beaches MLS,RX-11068190,N,Y,26.919651,-80.066425,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,Mary Martire,Illustrated Properties LLC (Jupiter),None,None,Jeremy Sitton,Sutter & Nugent LLC,None,None,6,blue,Jupiter Ocean Grande 1 Condo at 131 Ocean Gran...
6,PAST SALE,2025-05-21,Condo/Co-op,1400 S Ocean Blvd Unit 104n,Boca Raton,FL,33432.0,1555000.0,2.0,2.5,Addison on The Ocean,2156.0,NaN,1986.0,NaN,721.0,3348.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Beaches MLS,RX-11018830,N,Y,26.363561,-80.068653,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Bonnie Heatzig,Compass Florida LLC,None,None,Lisa Margolin,Realty Home Advisors Inc,None,None,7,blue,Addison on The Ocean at 1400 S Ocean Blvd Unit...
7,PAST SALE,2025-05-20,Condo/Co-op,102 Palm Bay Cir Unit C,Palm Beach Gardens,FL,33418.0,1275000.0,3.0,3.0,Palm Bay Club Condo,2483.0,43560.0,2003.0,NaN,513.0,1637.0,So

In [60]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Number One North Ocean Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,150,000 to $7,900,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Palm Beach, Jupiter, Boca Raton, Palm Beach Gardens, Singer Island, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 85 condo sales totaling $48,469,925 million from May. 17th to May. 25th. The previous week, brokers closed 97 condo sales totaling $69,559,250.

Last week’s units sold for an average of $570,234, lower than the $717,106 a

In [61]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [62]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [63]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [64]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-22,Condo/Co-op,150 N Ocean Blvd #403,Palm Beach,FL,33480.0,7900000.0,3.0,4.5,Number One North Ocean Condo,3036.0,NaN,1980.0,NaN,2602.0,6566.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Beaches MLS,RX-11076066,N,Y,26.720386,-80.034950,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Christopher Leavitt,Douglas Elliman,None,None,Lawrence Moens,Lawrence A. Moens Associates,None,None,1,orange,Number One North Ocean Condo at 150 N Ocean Bl...
1,PAST SALE,2025-05-21,Condo/Co-op,200 SE Mizner Blvd #507,Boca Raton,FL,33432.0,3501000.0,2.0,3.5,Alina Boca Raton Condo,2713.0,NaN,2021.0,NaN,1290.0,3216.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11049060,N,Y,26.347715,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Rochelle LeCavalier,Douglas Elliman,None,None,2,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
2,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 201n,Palm Beach,FL,33480.0,3100000.0,2.0,2.5,Beach Point Condo,2137.0,NaN,1979.0,NaN,1451.0,2799.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11048433,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Thor Brown,Compass Florida LLC (PB),None,None,Ashley Copeland,Brown Harris Stevens of PB,None,None,3,blue,Beach Point Condo at 2660 S Ocean Blvd Unit 20...
3,PAST SALE,2025-05-21,Condo/Co-op,3730 N Ocean Dr Unit 11d,Singer Island,FL,33404.0,2995000.0,3.0,3.0,Vistablue,2686.0,NaN,2018.0,NaN,1115.0,3645.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Beaches MLS,RX-11019236,N,Y,26.791736,-80.033375,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Judy Ballard,Illustrated Properties LLC (Co,None,None,John Pinkham,Exceptional Properties,None,None,4,blue,Vistablue at 3730 N Ocean Dr Unit 11d in Singe...
4,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 402s,Palm Beach,FL,33480.0,2190000.0,2.0,2.0,Beach Point Cond Decl Filed 5-31-79,1900.0,NaN,1979.0,NaN,1153.0,2149.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11068446,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Scott Gordon,Douglas Elliman,None,None,5,blue,Beach Point Cond Decl Filed 5-31-79 at 2660 S ...
5,PAST SALE,2025-05-20,Condo/Co-op,131 Ocean Grande Blvd #808,Jupiter,FL,33477.0,2050000.0,3.0,3.5,Jupiter Ocean Grande 1 Condo,2376.0,NaN,2001.0,NaN,863.0,2283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,Beaches MLS,RX-11068190,N,Y,26.919651,-80.066425,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,Mary Martire,Illustrated Properties LLC (Jupiter),None,None,Jeremy Sitton,Sutter & Nugent LLC,None,None,6,blue,Jupiter Ocean Grande 1 Condo at 131 Ocean Gran...
6,PAST SALE,2025-05-21,Condo/Co-op,1400 S Ocean Blvd Unit 104n,Boca Raton,FL,33432.0,1555000.0,2.0,2.5,Addison on The Ocean,2156.0,NaN,1986.0,NaN,721.0,3348.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Beaches MLS,RX-11018830,N,Y,26.363561,-80.068653,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Bonnie Heatzig,Compass Florida LLC,None,None,Lisa Margolin,Realty Home Advisors Inc,None,None,7,blue,Addison on The Ocean at 1400 S Ocean Blvd Unit...
7,PAST SALE,2025-05-20,Condo/Co-op,102 Palm Bay Cir Unit C,Palm Beach Gardens,FL,33418.0,1275000.0,3.0,3.0,Palm Bay Club Condo,2483.0,43560.0,2003.0,NaN,513.0,1637.0,So

In [65]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Number One North Ocean Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,150,000 to $7,900,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Palm Beach, Jupiter, Boca Raton, Palm Beach Gardens, Singer Island, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 85 condo sales totaling $48,469,925 million from May. 17th to May. 25th. The previous week, brokers closed 97 condo sales totaling $69,559,250.

Last week’s units sold for an average of $570,234, lower than the $717,106 a

In [73]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Palm-Beach/150-N-Ocean-Blvd-33480/unit-403/home/195194539


In [74]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Boca-Raton/200-SE-Mizner-Blvd-33432/unit-507/home/147297126


In [75]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Riviera-Beach/3000-N-Ocean-Dr-33404/unit-23C/home/42463091


In [76]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Palm-Beach-Gardens/104-Chambord-Ter-33410/home/42592431


In [77]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/150-N-Ocean-Blvd-33480/unit-403/home/195194539


In [78]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Palm-Beach-Gardens/104-Chambord-Ter-33410/home/42592431


In [79]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-22,Condo/Co-op,150 N Ocean Blvd #403,Palm Beach,FL,33480.0,7900000.0,3.0,4.5,Number One North Ocean Condo,3036.0,NaN,1980.0,NaN,2602.0,6566.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Beaches MLS,RX-11076066,N,Y,26.720386,-80.034950,https://www.redfin.com/FL/Palm-Beach/150-N-Oce...,Christopher Leavitt,Douglas Elliman,None,None,Lawrence Moens,Lawrence A. Moens Associates,None,None,1,orange,Number One North Ocean Condo at 150 N Ocean Bl...
1,PAST SALE,2025-05-21,Condo/Co-op,200 SE Mizner Blvd #507,Boca Raton,FL,33432.0,3501000.0,2.0,3.5,Alina Boca Raton Condo,2713.0,NaN,2021.0,NaN,1290.0,3216.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11049060,N,Y,26.347715,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Rochelle LeCavalier,Douglas Elliman,None,None,2,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
2,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 201n,Palm Beach,FL,33480.0,3100000.0,2.0,2.5,Beach Point Condo,2137.0,NaN,1979.0,NaN,1451.0,2799.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11048433,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Thor Brown,Compass Florida LLC (PB),None,None,Ashley Copeland,Brown Harris Stevens of PB,None,None,3,blue,Beach Point Condo at 2660 S Ocean Blvd Unit 20...
3,PAST SALE,2025-05-21,Condo/Co-op,3730 N Ocean Dr Unit 11d,Singer Island,FL,33404.0,2995000.0,3.0,3.0,Vistablue,2686.0,NaN,2018.0,NaN,1115.0,3645.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Beaches MLS,RX-11019236,N,Y,26.791736,-80.033375,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Judy Ballard,Illustrated Properties LLC (Co,None,None,John Pinkham,Exceptional Properties,None,None,4,blue,Vistablue at 3730 N Ocean Dr Unit 11d in Singe...
4,PAST SALE,2025-05-20,Condo/Co-op,2660 S Ocean Blvd Unit 402s,Palm Beach,FL,33480.0,2190000.0,2.0,2.0,Beach Point Cond Decl Filed 5-31-79,1900.0,NaN,1979.0,NaN,1153.0,2149.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11068446,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Scott Gordon,Douglas Elliman,None,None,5,blue,Beach Point Cond Decl Filed 5-31-79 at 2660 S ...
5,PAST SALE,2025-05-20,Condo/Co-op,131 Ocean Grande Blvd #808,Jupiter,FL,33477.0,2050000.0,3.0,3.5,Jupiter Ocean Grande 1 Condo,2376.0,NaN,2001.0,NaN,863.0,2283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,Beaches MLS,RX-11068190,N,Y,26.919651,-80.066425,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,Mary Martire,Illustrated Properties LLC (Jupiter),None,None,Jeremy Sitton,Sutter & Nugent LLC,None,None,6,blue,Jupiter Ocean Grande 1 Condo at 131 Ocean Gran...
6,PAST SALE,2025-05-21,Condo/Co-op,1400 S Ocean Blvd Unit 104n,Boca Raton,FL,33432.0,1555000.0,2.0,2.5,Addison on The Ocean,2156.0,NaN,1986.0,NaN,721.0,3348.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Beaches MLS,RX-11018830,N,Y,26.363561,-80.068653,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Bonnie Heatzig,Compass Florida LLC,None,None,Lisa Margolin,Realty Home Advisors Inc,None,None,7,blue,Addison on The Ocean at 1400 S Ocean Blvd Unit...
7,PAST SALE,2025-05-20,Condo/Co-op,102 Palm Bay Cir Unit C,Palm Beach Gardens,FL,33418.0,1275000.0,3.0,3.0,Palm Bay Club Condo,2483.0,43560.0,2003.0,NaN,513.0,1637.0,So

## Time on Market Calculator

In [81]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 1, 6) ## List (Earlier) date
date2 = datetime(2025, 5, 21) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

135


## Clean CSV for Datawrapper Chart

In [82]:
chart_df = df_top_ten

In [83]:
chart_df = chart_df.fillna(" ")

In [84]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [85]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [86]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [87]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [88]:
chart_df['$/SQUARE FEET'].astype(float)

0    2602.0
1    1290.0
2    1451.0
3    1115.0
4    1153.0
5     863.0
6     721.0
7     513.0
8     406.0
9     719.0
Name: $/SQUARE FEET, dtype: float64

In [89]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [90]:
csv_date_string = today.strftime("%m_%d_%Y")

In [91]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [92]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-05-22,150 N Ocean Blvd #403 in Palm Beach,"$7,900,000",3,4.5,"3,036",1980,"$2,602",Beaches MLS,RX-11076066,Christopher Leavitt Douglas Elliman,Lawrence Moens Lawrence A. Moens Associates
1,2025-05-21,200 SE Mizner Blvd #507 in Boca Raton,"$3,501,000",2,3.5,"2,713",2021,"$1,290",Beaches MLS,RX-11049060,Rachelle Hirt-Beresh Compass Florida LLC Scott...,Rochelle LeCavalier Douglas Elliman
2,2025-05-20,2660 S Ocean Blvd Unit 201n in Palm Beach,"$3,100,000",2,2.5,"2,137",1979,"$1,451",Beaches MLS,RX-11048433,Thor Brown Compass Florida LLC (PB),Ashley Copeland Brown Harris Stevens of PB
3,2025-05-21,3730 N Ocean Dr Unit 11d in Singer Island,"$2,995,000",3,3.0,"2,686",2018,"$1,115",Beaches MLS,RX-11019236,Judy Ballard Illustrated Properties LLC (Co,John Pinkham Exceptional Properties
4,2025-05-20,2660 S Ocean Blvd Unit 402s in Palm Beach,"$2,190,000",2,2.0,"1,900",1979,"$1,153",Beaches MLS,RX-11068446,"Fern Fodiman Sotheby's Intl. Realty, Inc.",Scott Gordon Douglas Elliman
5,2025-05-20,131 Ocean Grande Blvd #808 in Jupiter,"$2,050,000",3,3.5,"2,376",2001,$863,Beaches MLS,RX-11068190,Mary Martire Illustrated Properties LLC (Jupit...,Jeremy Sitton Sutter & Nugent LLC
6,2025-05-21,1400 S Ocean Blvd Unit 104n in Boca Raton,"$1,555,000",2,2.5,"2,156",1986,$721,Beaches MLS,RX-11018830,Bonnie Heatzig Compass Florida LLC,Lisa Margolin Realty Home Advisors Inc
7,2025-05-20,102 Palm Bay Cir Unit C in Palm Beach Gardens,"$1,275,000",3,3.0,"2,483",2003,$513,Beaches MLS,RX-11063852,Todd Blair Palm Beach Coastal Realty LLC,Laura Cole Illustrated Properties LLC (Co
8,2025-05-20,104 Chambord Ter Ter in Palm Beach Gardens,"$1,200,000",2,2.5,"2,953",2005,$406,Beaches MLS,RX-11067653,Richard Wood Frankel Ball Realty LLC Wendy Moy...,Richard Wood Frankel Ball Realty LLC
9,2025-05-21,3000 N Ocean Dr Unit 23-C in Singer Island,"$1,150,000",2,2.0,"1,600",1977,$719,Beaches MLS,RX-11078403,Matthew Brannon Illustrated Properties LLC (Co...,Susan Bennett The Keyes Company (PBG) Leah Tat...


In [93]:
df2 = df2.sort_values(by='PRICE',ascending=False).head(20)

In [94]:
df2.to_csv("PBC_last_week_sales.csv")